In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/mai-ml-contest-1/ex.csv
/kaggle/input/mai-ml-contest-1/train.csv
/kaggle/input/mai-ml-contest-1/test.csv


In [2]:
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix, hstack
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import StandardScaler
from tqdm import tqdm_notebook
from sklearn.model_selection import train_test_split

%matplotlib inline
import seaborn as sns
from matplotlib import pyplot as plt
from scipy.stats import norm, skew #for some statistics
from scipy import stats

In [3]:
# загрузим обучающую и тестовую выборки
train_df = pd.read_csv("/kaggle/input/mai-ml-contest-1/train.csv")
X_test = pd.read_csv("/kaggle/input/mai-ml-contest-1/test.csv")
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11017 entries, 0 to 11016
Data columns (total 36 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   ApplicationDate             10487 non-null  object 
 1   Age                         10487 non-null  float64
 2   AnnualIncome                10487 non-null  float64
 3   CreditScore                 9986 non-null   float64
 4   LoanAmount                  9986 non-null   float64
 5   LoanDuration                10487 non-null  float64
 6   MaritalStatus               10487 non-null  object 
 7   NumberOfDependents          10487 non-null  float64
 8   HomeOwnershipStatus         10487 non-null  object 
 9   MonthlyDebtPayments         9986 non-null   float64
 10  CreditCardUtilizationRate   10487 non-null  float64
 11  NumberOfOpenCreditLines     10487 non-null  float64
 12  NumberOfCreditInquiries     10487 non-null  float64
 13  DebtToIncomeRatio           104

In [4]:
pd.set_option('display.max_columns', None)  # Отобразить все столбцы
pd.set_option('display.expand_frame_repr', False)  # Не переносить строки

In [5]:
df_cleaned = train_df.dropna(how='all').reset_index(drop=True)
print("DataFrame после удаления строк, где только значения NaN:")
df_cleaned
train_df = df_cleaned

DataFrame после удаления строк, где только значения NaN:


In [6]:
train_df = train_df[(train_df['RiskScore'] >= -100000) & (train_df['RiskScore'] <= 100000)]

In [7]:
df_cleaned = train_df.dropna()
train_df = df_cleaned

In [8]:
train_df['ApplicationDate'] = pd.to_datetime(train_df['ApplicationDate'])
train_df['ApplicationDate'] = train_df['ApplicationDate'].apply(lambda ts: ts.year)
X_test['ApplicationDate'] = pd.to_datetime(X_test['ApplicationDate'])
X_test['ApplicationDate'] = X_test['ApplicationDate'].apply(lambda ts: ts.year)
train_df.describe()

,ApplicationDate,Age,AnnualIncome,CreditScore,LoanAmount,LoanDuration,NumberOfDependents,MonthlyDebtPayments,CreditCardUtilizationRate,NumberOfOpenCreditLines,NumberOfCreditInquiries,DebtToIncomeRatio,BankruptcyHistory,PreviousLoanDefaults,PaymentHistory,LengthOfCreditHistory,SavingsAccountBalance,CheckingAccountBalance,TotalAssets,TotalLiabilities,MonthlyIncome,UtilityBillsPaymentHistory,JobTenure,Experience,NetWorth,BaseInterestRate,InterestRate,MonthlyLoanPayment,TotalDebtToIncomeRatio,LoanApproved,RiskScore
count,9787.000000,9787.000000,9787.000000,9787.000000,9787.000000,9787.000000,9787.000000,9787.000000,9787.000000,9787.000000,9787.000000,9787.000000,9787.000000,9787.000000,9787.000000,9787.000000,9787.000000,9787.000000,9.787000e+03,9.787000e+03,9787.000000,9787.000000,9787.000000,9787.000000,9.787000e+03,9787.000000,9787.000000,9787.000000,9787.000000,9787.000000,9787.000000
mean,1998.947686,39.898743,131527.784510,677.911515,29925.122509,53.439052,1.568714,546.437928,0.284935,3.029733,0.976704,0.249999,0.052621,0.096250,24.011035,15.615306,5512.410749,1987.982323,1.823864e+05,6.875512e+04,10831.569727,0.784073,4.938694,17.671299,1.536230e+05,0.200502,0.200155,1079.984544,0.519815,0.512108,48.349800
std,11.836111,11.606484,116069.069481,175.374136,27812.837524,24.487196,1.416543,502.599753,0.159821,1.740653,0.986558,0.144407,0.223287,0.294949,4.849554,8.675042,8513.766741,3063.971711,4.665750e+05,2.012566e+05,9398.941396,0.123399,2.202112,11.336445,4.613971e+05,0.094560,0.096607,1355.683134,0.899259,0.499879,17.255990
min,1979.000000,18.000000,15000.000000,300.000000,1063.000000,12.000000,0.000000,13.000000,0.003674,0.000000,0.000000,0.003227,0.000000,0.000000,8.000000,1.000000,31.000000,16.000000,8.920000e+02,7.800000e+01,1250.000000,0.259301,0.000000,0.000000,1.004000e+03,0.052494,0.046445,30.008506,0.006064,0.000000,14.841417
25%,1989.000000,32.000000,20699.500000,550.000000,12656.000000,36.000000,0.000000,233.000000,0.159490,2.000000,0.000000,0.138296,0.000000,0.000000,21.000000,8.000000,1405.000000,532.000000,2.308400e+04,7.983500e+03,1666.750000,0.707881,3.000000,9.000000,7.253000e+03,0.119862,0.119548,374.944006,0.066858,0.000000,32.699940
50%,1999.000000,40.000000,88615.000000,722.000000,21836.000000,48.000000,1.000000,398.000000,0.262798,3.000000,1.000000,0.227809,0.000000,0.000000,24.000000,16.000000,2987.000000,1118.000000,6.036200e+04,2.267900e+04,7339.916667,0.803716,5.000000,17.000000,2.724500e+04,0.182072,0.180496,685.742578,0.178283,1.000000,44.151002
75%,2009.000000,48.000000,257780.500000,850.000000,37289.500000,60.000000,3.000000,683.500000,0.391759,4.000000,2.000000,0.340077,0.000000,0.000000,27.000000,23.000000,6273.500000,2297.000000,1.611985e+05,6.012250e+04,21384.000000,0.878835,6.000000,26.000000,1.239920e+05,0.265098,0.265152,1285.051520,0.641459,1.000000,65.122034
max,2020.000000,80.000000,748508.000000,850.000000,418997.000000,120.000000,6.000000,10879.000000,0.914635,12.000000,6.000000,0.865842,1.000000,1.000000,44.000000,30.000000,222556.000000,109245.000000,1.128946e+07,8.997054e+06,25000.000000,0.996573,17.000000,57.000000,1.126117e+07,0.722497,0.833647,29634.807816,24.383046,1.000000,97.597249


In [9]:

train_df['MaritalStatus'] = train_df['MaritalStatus'].astype('category')
mean_target_per_category = train_df.groupby('MaritalStatus')['LoanApproved'].mean()
train_df['MaritalStatus_TargetEncoded'] = train_df['MaritalStatus'].map(mean_target_per_category)

X_test['MaritalStatus'] = X_test['MaritalStatus'].astype('category')
mean_target_per_category = X_test.groupby('MaritalStatus')['LoanApproved'].mean()
X_test['MaritalStatus_TargetEncoded'] = X_test['MaritalStatus'].map(mean_target_per_category)


train_df = pd.get_dummies(train_df, columns=['MaritalStatus'], prefix='Mar')
train_df = train_df.drop(columns=['Mar_Widowed'])


train_df = pd.get_dummies(train_df, columns=['EmploymentStatus'], prefix='Emp')
train_df = train_df.drop(columns=['Emp_Unemployed'])



X_test = pd.get_dummies(X_test, columns=['MaritalStatus'], prefix='Mar')
X_test = X_test.drop(columns=['Mar_Widowed'])


X_test = pd.get_dummies(X_test, columns=['EmploymentStatus'], prefix='Emp')
X_test = X_test.drop(columns=['Emp_Unemployed'])
train_df


train_df = pd.get_dummies(train_df, columns=['HomeOwnershipStatus'], drop_first=True)
marital_status_columns = [col for col in train_df.columns if 'HomeOwnershipStatus_' in col]
for marital_col in marital_status_columns:
    train_df[f'ApprovalRate_{marital_col}'] = train_df.groupby(marital_col)['LoanApproved'].transform('mean')

X_test = pd.get_dummies(X_test, columns=['HomeOwnershipStatus'], drop_first=True)
marital_status_columns = [col for col in X_test.columns if 'HomeOwnershipStatus_' in col]
for marital_col in marital_status_columns:
    X_test[f'ApprovalRate_{marital_col}'] = X_test.groupby(marital_col)['LoanApproved'].transform('mean')

    

train_df



/tmp/ipykernel_17/752407492.py:17: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  mean_target_per_category = train_df.groupby('MaritalStatus')['LoanApproved'].mean()
/tmp/ipykernel_17/752407492.py:21: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  mean_target_per_category = X_test.groupby('MaritalStatus')['LoanApproved'].mean()


,ApplicationDate,Age,AnnualIncome,CreditScore,LoanAmount,LoanDuration,NumberOfDependents,MonthlyDebtPayments,CreditCardUtilizationRate,NumberOfOpenCreditLines,NumberOfCreditInquiries,DebtToIncomeRatio,BankruptcyHistory,LoanPurpose,PreviousLoanDefaults,PaymentHistory,LengthOfCreditHistory,SavingsAccountBalance,CheckingAccountBalance,TotalAssets,TotalLiabilities,MonthlyIncome,UtilityBillsPaymentHistory,JobTenure,EducationLevel,Experience,NetWorth,BaseInterestRate,InterestRate,MonthlyLoanPayment,TotalDebtToIncomeRatio,LoanApproved,RiskScore,MaritalStatus_TargetEncoded,Mar_Divorced,Mar_Married,Mar_Single,Emp_Employed,Emp_Self-Employed,HomeOwnershipStatus_Other,HomeOwnershipStatus_Own,HomeOwnershipStatus_Rent,ApprovalRate_HomeOwnershipStatus_Other,ApprovalRate_HomeOwnershipStatus_Own,ApprovalRate_HomeOwnershipStatus_Rent
0,2010,27.0,66829.0,549.0,17290.0,60.0,1.0,1095.0,0.151985,2.0,1.0,0.168542,0.0,Education,0.0,23.0,5.0,2539.0,247.0,50072.0,15005.0,5569.083333,0.908289,4.0,Associate,4.0,35067.0,0.257790,0.251465,508.970230,0.288013,0.0,66.176500,0.536835,True,False,False,True,False,False,False,True,0.512812,0.508844,0.499156
1,1996,55.0,172147.0,850.0,16110.0,36.0,1.0,211.0,0.175693,3.0,0.0,0.141553,0.0,Home,0.0,27.0,11.0,844.0,694.0,91878.0,64877.0,14345.583333,0.797339,2.0,High School,33.0,27001.0,0.086110,0.093173,514.675859,0.050585,1.0,28.495737,0.515385,False,False,False,True,False,False,False,False,0.512812,0.508844,0.517732
2,2015,51.0,300000.0,850.0,38436.0,36.0,0.0,546.0,0.444605,3.0,0.0,0.030987,1.0,Education,0.0,21.0,12.0,12756.0,651.0,292300.0,13918.0,25000.000000,0.915945,3.0,Bachelor,28.0,278382.0,0.108436,0.115443,1268.276385,0.072571,1.0,34.488104,0.506247,False,True,False,True,False,False,False,False,0.512812,0.508844,0.517732
3,1981,25.0,34683.0,847.0,19186.0,48.0,0.0,153.0,0.188452,7.0,0.0,0.379843,0.0,Debt Consolidation,0.0,27.0,21.0,1524.0,529.0,13070.0,3846.0,2890.250000,0.876416,3.0,High School,0.0,9224.0,0.100686,0.112822,498.505187,0.225415,1.0,36.910753,0.506247,False,True,False,True,False,True,False,False,0.505688,0.508844,0.517732
4,1995,55.0,300000.0,850.0,30437.0,48.0,2.0,562.0,0.273431,3.0,1.0,0.068659,0.0,Debt Consolidation,0.0,35.0,9.0,1973.0,128.0,33357.0,351603.0,25000.000000,0.888689,5.0,Bachelor,31.0,4502.0,0.110437,0.089037,756.035156,0.052721,1.0,31.347091,0.508836,False,False,True,True,False,False,False,True,0.512812,0.508844,0.499156
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10481,1984,47.0,88686.0,648.0,5472.0,12.0,2.0,98.0,0.112650,8.0,1.0,0.311830,0.0,Home,0.0,25.0,16.0,7044.0,3957.0,143195.0,21497.0,7390.500000,0.694441,4.0,Bachelor,23.0,121698.0,0.156472,0.164928,497.756143,0.080611,1.0,41.553066,0.536835,True,False,False,True,False,False,False,True,0.512812,0.508844,0.499156
10482,1980,39.0,300000.0,850.0,7721.0,72.0,5.0,276.0,0.215136,2.0,1.0,0.165864,0.0,Home,0.0,29.0,8.0,766.0,695.0,64283.0,45676.0,25000.000000,0.877998,2.0,Associate,16.0,18607.0,0.107721,0.108596,146.407421,0.016896,1.0,28.051757,0.536835,True,False,False,True,False,False,True,False,0.512812,0.524937,0.517732
10483,2010,57.0,300000.0,850.0,14756.0,48.0,2.0,620.0,0.477742,1.0,3.0,0.154680,0.0,Education,0.0,26.0,2.0,2064.0,798.0,168225.0,681.0,25000.000000,0.786244,4.0,Associate,38.0,167544.0,0.094756,0.106387,378.792448,0.039952,1.0,32.440586,0.506247,False,True,False,True,False,False,True,False,0.512812,0.524937,0.517732
10484,2003,28.0,15000.0,300.0,51361.0,96.0,1.0,696.0,0.322554,2.0,0.0,0.175333,0.0,Home,0.0,24.0,2.0,4418.0,3846.0,118627.0,1748.0,1250.000000,0.738173,3.0,Bachelor,5.0,116879.0,0.446361,0.401087,1793.042130,1.991234,0.0,67.673170,0.508836,False,False,True,True,False,False,True,False,0.512812,0.524937,0.517732


In [10]:
train_df['AssetsToLoanApproval'] = train_df['TotalAssets'] / (train_df['LoanApproved'] + 1)


train_df = pd.get_dummies(train_df, columns=['LoanPurpose'], drop_first=True)
loan_purpose_columns = [col for col in train_df.columns if 'LoanPurpose_' in col]
for purpose_col in loan_purpose_columns:
    train_df[f'ApprovalRate_{purpose_col}'] = train_df.groupby(purpose_col)['LoanApproved'].transform('mean')

train_df = pd.get_dummies(train_df, columns=['EducationLevel'], drop_first=True)
education_columns = [col for col in train_df.columns if 'EducationLevel_' in col]
for edu_col in education_columns:
    train_df[f'ApprovalRate_{edu_col}'] = train_df.groupby(edu_col)['LoanApproved'].transform('mean')

train_df['ApprovedLoanAmount'] = train_df['LoanAmount'] * train_df['LoanApproved']


median_loan_amount = train_df['LoanAmount'].median()
train_df['HighLoanAmountApproved'] = ((train_df['LoanAmount'] > median_loan_amount) & (train_df['LoanApproved'] == 1)).astype(float)



X_test['AssetsToLoanApproval'] = X_test['TotalAssets'] / (X_test['LoanApproved'] + 1)

X_test = pd.get_dummies(X_test, columns=['LoanPurpose'], drop_first=True)
loan_purpose_columns = [col for col in X_test.columns if 'LoanPurpose_' in col]
for purpose_col in loan_purpose_columns:
    X_test[f'ApprovalRate_{purpose_col}'] = X_test.groupby(purpose_col)['LoanApproved'].transform('mean')

X_test = pd.get_dummies(X_test, columns=['EducationLevel'], drop_first=True)
education_columns = [col for col in X_test.columns if 'EducationLevel_' in col]
for edu_col in education_columns:
    X_test[f'ApprovalRate_{edu_col}'] = X_test.groupby(edu_col)['LoanApproved'].transform('mean')


X_test['ApprovedLoanAmount'] = X_test['LoanAmount'] * X_test['LoanApproved']

median_loan_amount = X_test['LoanAmount'].median() 
X_test['HighLoanAmountApproved'] = ((X_test['LoanAmount'] > median_loan_amount) & (X_test['LoanApproved'] == 1)).astype(float)



median_loan_amount

22289.0

In [11]:

train_df['GoodCreditRejected'] = ((train_df['CreditScore'] > 650) & (train_df['LoanApproved'] == 0)).astype(float)
train_df['HighIncomeRejected'] = ((train_df['AnnualIncome'] > 80000) & (train_df['LoanApproved'] == 0)).astype(float)
train_df['LowDebtRejected'] = ((train_df['DebtToIncomeRatio'] < 0.3) & (train_df['LoanApproved'] == 0)).astype(float)
train_df['LongCreditHistoryRejected'] = ((train_df['LengthOfCreditHistory'] > 4) & (train_df['LoanApproved'] == 0)).astype(float)
train_df['HomeOwnerRejected'] = ((train_df['HomeOwnershipStatus_Own'] == True) & (train_df['LoanApproved'] == 0)).astype(float)
train_df['HighSavingsRejected'] = ((train_df['SavingsAccountBalance'] > 20000) & (train_df['LoanApproved'] == 0)).astype(float)
train_df['HighAssetsRejected'] = ((train_df['TotalAssets'] > 110000) & (train_df['LoanApproved'] == 0)).astype(float)
train_df['HighNetWorthRejected'] = ((train_df['NetWorth'] > 100000) & (train_df['LoanApproved'] == 0)).astype(float)
train_df['LowCreditUtilizationRejected'] = ((train_df['CreditCardUtilizationRate'] < 0.2) & (train_df['LoanApproved'] == 0)).astype(float)
train_df['NoBankruptcyRejected'] = ((train_df['BankruptcyHistory'] == 0) & (train_df['LoanApproved'] == 0)).astype(float)
train_df['GoodPaymentHistoryRejected'] = ((train_df['PaymentHistory'] == 1) & (train_df['LoanApproved'] == 0)).astype(float)
train_df['HighMonthlyIncomeRejected'] = ((train_df['MonthlyIncome'] > 8500) & (train_df['LoanApproved'] == 0)).astype(float) 
train_df['NoPreviousDefaultsRejected'] = ((train_df['PreviousLoanDefaults'] == 0) & (train_df['LoanApproved'] == 0)).astype(float)
train_df['LowDebtToIncomeRatioRejected'] = ((train_df['DebtToIncomeRatio'] < 0.2) & (train_df['LoanApproved'] == 0)).astype(float) 
train_df['GoodUtilityBillsPaymentHistoryRejected'] = ((train_df['UtilityBillsPaymentHistory'] == 1) & (train_df['LoanApproved'] == 0)).astype(float)
train_df['StableEmploymentRejected'] = ((train_df['Emp_Employed'] == True) & (train_df['LoanApproved'] == 0)).astype(float)
train_df['NoCreditDefaultsRejected'] = ((train_df['PreviousLoanDefaults'] == 0) & (train_df['LoanApproved'] == 0)).astype(float)
train_df['LowMonthlyDebtRejected'] = ((train_df['MonthlyDebtPayments'] < 600) & (train_df['LoanApproved'] == 0)).astype(float)
train_df['GoodEducationRejected'] = (((train_df['EducationLevel_Doctorate'] == True) | (train_df['EducationLevel_Master'] == True)) & (train_df['LoanApproved'] == 0)).astype(float)
train_df['HighCheckingBalanceRejected'] = ((train_df['CheckingAccountBalance'] > 4000) & (train_df['LoanApproved'] == 0)).astype(float) 
train_df['HighTotalAssetsRejected'] = ((train_df['TotalAssets'] > 400000) & (train_df['LoanApproved'] == 0)).astype(float)
train_df['GoodCreditHistoryRejected'] = ((train_df['LengthOfCreditHistory'] > 18) & (train_df['LoanApproved'] == 0)).astype(float)
train_df['HighLoanAmountRejected'] = ((train_df['LoanAmount'] > 8000) & (train_df['LoanApproved'] == 0)).astype(float)
train_df['HighCreditApproved'] = ((train_df['CreditScore'] > 800) & (train_df['LoanApproved'] == 1)).astype(float)





X_test['GoodCreditRejected'] = ((X_test['CreditScore'] > 650) & (X_test['LoanApproved'] == 0)).astype(float)
X_test['HighIncomeRejected'] = ((X_test['AnnualIncome'] > 80000) & (X_test['LoanApproved'] == 0)).astype(float)
X_test['LowDebtRejected'] = ((X_test['DebtToIncomeRatio'] < 0.3) & (X_test['LoanApproved'] == 0)).astype(float)
X_test['LongCreditHistoryRejected'] = ((X_test['LengthOfCreditHistory'] > 4) & (X_test['LoanApproved'] == 0)).astype(float)
X_test['HomeOwnerRejected'] = ((X_test['HomeOwnershipStatus_Own'] == True) & (X_test['LoanApproved'] == 0)).astype(float)
X_test['HighSavingsRejected'] = ((X_test['SavingsAccountBalance'] > 20000) & (X_test['LoanApproved'] == 0)).astype(float)
X_test['HighAssetsRejected'] = ((X_test['TotalAssets'] > 110000) & (X_test['LoanApproved'] == 0)).astype(float)
X_test['HighNetWorthRejected'] = ((X_test['NetWorth'] > 100000) & (X_test['LoanApproved'] == 0)).astype(float)
X_test['LowCreditUtilizationRejected'] = ((X_test['CreditCardUtilizationRate'] < 0.2) & (X_test['LoanApproved'] == 0)).astype(float)
X_test['NoBankruptcyRejected'] = ((X_test['BankruptcyHistory'] == 0) & (X_test['LoanApproved'] == 0)).astype(float)
X_test['GoodPaymentHistoryRejected'] = ((X_test['PaymentHistory'] == 1) & (X_test['LoanApproved'] == 0)).astype(float)
X_test['HighMonthlyIncomeRejected'] = ((X_test['MonthlyIncome'] > 85000) & (X_test['LoanApproved'] == 0)).astype(float)
X_test['NoPreviousDefaultsRejected'] = ((X_test['PreviousLoanDefaults'] == 0) & (X_test['LoanApproved'] == 0)).astype(float)
X_test['LowDebtToIncomeRatioRejected'] = ((X_test['DebtToIncomeRatio'] < 0.2) & (X_test['LoanApproved'] == 0)).astype(float)
X_test['GoodUtilityBillsPaymentHistoryRejected'] = ((X_test['UtilityBillsPaymentHistory'] == 1) & (X_test['LoanApproved'] == 0)).astype(float)
X_test['StableEmploymentRejected'] = ((X_test['Emp_Employed'] == True) & (X_test['LoanApproved'] == 0)).astype(float)
X_test['NoCreditDefaultsRejected'] = ((X_test['PreviousLoanDefaults'] == 0) & (X_test['LoanApproved'] == 0)).astype(float)
X_test['LowMonthlyDebtRejected'] = ((X_test['MonthlyDebtPayments'] < 600) & (X_test['LoanApproved'] == 0)).astype(float)
X_test['GoodEducationRejected'] = (((X_test['EducationLevel_Doctorate'] == True) | (X_test['EducationLevel_Master'] == True)) & (X_test['LoanApproved'] == 0)).astype(float)
X_test['HighCheckingBalanceRejected'] = ((X_test['CheckingAccountBalance'] > 4000) & (X_test['LoanApproved'] == 0)).astype(float)
X_test['HighTotalAssetsRejected'] = ((X_test['TotalAssets'] > 400000) & (X_test['LoanApproved'] == 0)).astype(float) 
X_test['GoodCreditHistoryRejected'] = ((X_test['LengthOfCreditHistory'] > 18) & (X_test['LoanApproved'] == 0)).astype(float)
X_test['HighLoanAmountRejected'] = ((X_test['LoanAmount'] > 8000) & (X_test['LoanApproved'] == 0)).astype(float)
X_test['HighCreditApproved'] = ((X_test['CreditScore'] > 800) & (X_test['LoanApproved'] == 1)).astype(float)


train_df




,ApplicationDate,Age,AnnualIncome,CreditScore,LoanAmount,LoanDuration,NumberOfDependents,MonthlyDebtPayments,CreditCardUtilizationRate,NumberOfOpenCreditLines,NumberOfCreditInquiries,DebtToIncomeRatio,BankruptcyHistory,PreviousLoanDefaults,PaymentHistory,LengthOfCreditHistory,SavingsAccountBalance,CheckingAccountBalance,TotalAssets,TotalLiabilities,MonthlyIncome,UtilityBillsPaymentHistory,JobTenure,Experience,NetWorth,BaseInterestRate,InterestRate,MonthlyLoanPayment,TotalDebtToIncomeRatio,LoanApproved,RiskScore,MaritalStatus_TargetEncoded,Mar_Divorced,Mar_Married,Mar_Single,Emp_Employed,Emp_Self-Employed,HomeOwnershipStatus_Other,HomeOwnershipStatus_Own,HomeOwnershipStatus_Rent,ApprovalRate_HomeOwnershipStatus_Other,ApprovalRate_HomeOwnershipStatus_Own,ApprovalRate_HomeOwnershipStatus_Rent,AssetsToLoanApproval,LoanPurpose_Debt Consolidation,LoanPurpose_Education,LoanPurpose_Home,LoanPurpose_Other,ApprovalRate_LoanPurpose_Debt Consolidation,ApprovalRate_LoanPurpose_Education,ApprovalRate_LoanPurpose_Home,ApprovalRate_LoanPurpose_Other,EducationLevel_Bachelor,EducationLevel_Doctorate,EducationLevel_High School,EducationLevel_Master,ApprovalRate_EducationLevel_Bachelor,ApprovalRate_EducationLevel_Doctorate,ApprovalRate_EducationLevel_High School,ApprovalRate_EducationLevel_Master,ApprovedLoanAmount,HighLoanAmountApproved,GoodCreditRejected,HighIncomeRejected,LowDebtRejected,LongCreditHistoryRejected,HomeOwnerRejected,HighSavingsRejected,HighAssetsRejected,HighNetWorthRejected,LowCreditUtilizationRejected,NoBankruptcyRejected,GoodPaymentHistoryRejected,HighMonthlyIncomeRejected,NoPreviousDefaultsRejected,LowDebtToIncomeRatioRejected,GoodUtilityBillsPaymentHistoryRejected,StableEmploymentRejected,NoCreditDefaultsRejected,LowMonthlyDebtRejected,GoodEducationRejected,HighCheckingBalanceRejected,HighTotalAssetsRejected,GoodCreditHistoryRejected,HighLoanAmountRejected,HighCreditApproved
0,2010,27.0,66829.0,549.0,17290.0,60.0,1.0,1095.0,0.151985,2.0,1.0,0.168542,0.0,0.0,23.0,5.0,2539.0,247.0,50072.0,15005.0,5569.083333,0.908289,4.0,4.0,35067.0,0.257790,0.251465,508.970230,0.288013,0.0,66.176500,0.536835,True,False,False,True,False,False,False,True,0.512812,0.508844,0.499156,50072.0,False,True,False,False,0.507182,0.489510,0.509319,0.513815,False,False,False,False,0.503476,0.508876,0.537145,0.506988,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,1996,55.0,172147.0,850.0,16110.0,36.0,1.0,211.0,0.175693,3.0,0.0,0.141553,0.0,0.0,27.0,11.0,844.0,694.0,91878.0,64877.0,14345.583333,0.797339,2.0,33.0,27001.0,0.086110,0.093173,514.675859,0.050585,1.0,28.495737,0.515385,False,False,False,True,False,False,False,False,0.512812,0.508844,0.517732,45939.0,False,False,True,False,0.507182,0.516435,0.518671,0.513815,False,False,True,False,0.503476,0.508876,0.475032,0.506988,16110.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,2015,51.0,300000.0,850.0,38436.0,36.0,0.0,546.0,0.444605,3.0,0.0,0.030987,1.0,0.0,21.0,12.0,12756.0,651.0,292300.0,13918.0,25000.000000,0.915945,3.0,28.0,278382.0,0.108436,0.115443,1268.276385,0.072571,1.0,34.488104,0.506247,False,True,False,True,False,False,False,False,0.512812,0.508844,0.517732,146150.0,False,True,False,False,0.507182,0.489510,0.509319,0.513815,True,False,False,False,0.532778,0.508876,0.537145,0.506988,38436.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,1981,25.0,34683.0,847.0,19186.0,48.0,0.0,153.0,0.188452,7.0,0.0,0.379843,0.0,0.0,27.0,21.0,1524.0,529.0,13070.0,3846.0,2890.250000,0.876416,3.0,0.0,9224.0,0.100686,0.112822,498.505187,0.225415,1.0,36.910753,0.506247,False,True,False,True,False,True,False,False,0.505688,0.508844,0.517732,6535.0,True,False,False,False,0.527212,0.516435,0.509319,0.513815,False,False,True,False,0.503476,0.508876,0.475032,0.506988,19186.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.

In [12]:
train_df['LowDebtToIncomeRatio'] = (train_df['TotalDebtToIncomeRatio'] < 0.4).astype(float)
train_df['SavingsToIncomeRatio'] = train_df['SavingsAccountBalance'] / train_df['AnnualIncome']

X_test['LowDebtToIncomeRatio'] = (X_test['TotalDebtToIncomeRatio'] < 0.4).astype(float)
X_test['SavingsToIncomeRatio'] = X_test['SavingsAccountBalance'] / X_test['AnnualIncome']

In [13]:


train_df['new'] = abs(train_df['CheckingAccountBalance'] - train_df['SavingsAccountBalance'])
train_df['neww'] = abs(train_df['MonthlyIncome'] - train_df['MonthlyDebtPayments'])
train_df['LiquidAssets'] = train_df['CheckingAccountBalance'] + train_df['SavingsAccountBalance']
train_df['newww'] = abs(train_df['TotalAssets'] - train_df['TotalLiabilities'])
train_df['LiquidAssetsToLiabilitiesRatio'] = train_df['LiquidAssets'] / train_df['TotalAssets']
train_df['NetWorthToLiabilitiesRatio'] = train_df['NetWorth'] / train_df['TotalLiabilities']
train_df['NetWorthCategory'] = pd.cut(train_df['NetWorth'], bins=[-float('inf'), 0, 50000, float('inf')], labels=[0.0, 1.0, 2.0])
train_df['NetWorthCategory'] = train_df['NetWorthCategory'].astype(float)
train_df['HighRisk'] = train_df['CreditScore'].apply(lambda x: 1 if x < 600 else 0)
train_df['IncomeCategory'] = pd.cut(train_df['AnnualIncome'], bins=[0, 30000, 60000, 90000, float('inf')], labels=[1.0, 2.0, 3.0, 4.0])
income_group_mean = train_df.groupby('IncomeCategory')['CreditScore'].transform('mean')
train_df['CreditScoreDeviationIncome'] = train_df['CreditScore'] - income_group_mean
train_df['DebtToAssetRatio'] = train_df['TotalLiabilities'] / train_df['TotalAssets']
train_df['MonthlyLoanPaymentToIncomeRatio'] = train_df['MonthlyLoanPayment'] / train_df['MonthlyIncome']
train_df['UtilityPaymentsToIncomeRatio'] = train_df['UtilityBillsPaymentHistory'] / train_df['MonthlyIncome']
train_df['EmploymentStability'] = train_df['JobTenure'].apply(lambda x: 1 if x >= 5 else 0)
train_df['CreditScoreCategory'] = pd.cut(train_df['CreditScore'], bins=[0, 300, 550, 722, 850, float('inf')], labels=[1.0, 2.0, 3.0, 4.0, 5.0])
train_df['HighCreditScoreFlag'] = train_df['CreditScore'].apply(lambda x: 1 if x >= 740 else 0)
train_df['DefaultRiskScore'] = train_df['CreditScore'].apply(lambda x: 1 if x < 580 else 0)
train_df['DefaultRiskScoren'] = train_df['CreditScore'].apply(lambda x: 1 if x < 550 else 0)
train_df['WeightedCreditScore'] = train_df['CreditScore'] * train_df['AnnualIncome']
train_df['IncomeToDebtRatio'] = train_df['AnnualIncome'] / train_df['TotalLiabilities']
hours_per_year = 2080  
train_df['HourlyIncome'] = train_df['AnnualIncome'] / hours_per_year
train_df['IncomeByAgeGroup'] = train_df.groupby('Age')['AnnualIncome'].transform('mean')
train_df['IncomeToBaseInterestRate'] = train_df['AnnualIncome'] / train_df['BaseInterestRate']
train_df['LoanApprovedFlag'] = (train_df['LoanApproved'] == 1).astype(float)
median_loan_amount = train_df['LoanAmount'].median()
train_df['HighLoanAmountFlag'] = (train_df['LoanAmount'] > median_loan_amount).astype(float)
train_df['HighLoanAmountApproval'] = train_df['LoanApproved'] * train_df['HighLoanAmountFlag']
train_df['IncomeToLoanApproval'] = train_df['AnnualIncome'] / (train_df['LoanApproved'] + 1)
train_df['MonthlyPaymentToApproval'] = train_df['MonthlyLoanPayment'] / (train_df['LoanApproved'] + 1)
train_df['DebtToIncomeCategory'] = pd.cut(train_df['DebtToIncomeRatio'], bins=[-float('inf'), 0.2, 0.3, 0.4, 0.499, float('inf')], labels=[0.0, 1.0, 2.0, 3.0, 4.0]) 
train_df['SavingsBalanceCategory'] = pd.cut(train_df['SavingsAccountBalance'], bins=[-float('inf'), 1000, 4000, 8000, float('inf')], labels=[0.0, 1.0, 2.0, 3.0]) 
train_df['LoanAmountCategory'] = pd.cut(train_df['LoanAmount'], bins=[0, 10000, 30000, 50000, 70000, float('inf')], labels=[0.0, 1.0, 2.0, 3.0, 4.0])  
train_df['TotalAssetsCategory'] = pd.cut(train_df['TotalAssets'], bins=[0, 50000, 150000, 300000, float('inf')], labels=[0.0, 1.0, 2.0, 3.0])  
train_df['CreditHistoryCategory'] = pd.cut(train_df['LengthOfCreditHistory'], bins=[0, 2, 5, 9, float('inf')], labels=[0.0, 1.0, 2.0, 3.0])
train_df['CreditCardUtilizationCategory'] = pd.cut(train_df['CreditCardUtilizationRate'], bins=[-float('inf'), 0.2, 0.3, 0.4, 0.5, float('inf')], labels=[0.0, 1.0, 2.0, 3.0, 4.0])
train_df['PaymentHistoryCategory'] = pd.cut(train_df['PaymentHistory'], bins=[-1, 14, float('inf')], labels=[0.0, 1.0])
train_df['LowDebtToIncomeFlag'] = (train_df['DebtToIncomeCategory'] <= 1.0).astype(float)
train_df['InterestRateToIncomeRatio'] = train_df['InterestRate'] / train_df['AnnualIncome']
train_df['InterestRateToIncomeCategory'] = pd.cut(train_df['InterestRateToIncomeRatio'], bins=[-float('inf'), 1e-7, 2e-7, 5e-6, 1e-5, float('inf')], labels=[0.0, 1.0, 2.0, 3.0, 4.0])
train_df['CheckingBalanceCategory'] = pd.cut(train_df['CheckingAccountBalance'], bins=[-float('inf'), 19000, 35000, 60000, float('inf')], labels=[0.0, 1.0, 2.0, 3.0])



X_test['new'] = abs(X_test['CheckingAccountBalance'] - X_test['SavingsAccountBalance'])
X_test['neww'] = abs(X_test['MonthlyIncome'] - X_test['MonthlyDebtPayments'])
X_test['LiquidAssets'] = X_test['CheckingAccountBalance'] + X_test['SavingsAccountBalance']
X_test['newww'] = abs(X_test['TotalAssets'] - X_test['TotalLiabilities'])
X_test['LiquidAssetsToLiabilitiesRatio'] = X_test['LiquidAssets'] / X_test['TotalAssets']
X_test['NetWorthToLiabilitiesRatio'] = X_test['NetWorth'] / X_test['TotalLiabilities']
X_test['NetWorthCategory'] = pd.cut(X_test['NetWorth'], bins=[-float('inf'), 0, 50000, float('inf')], labels=[0.0, 1.0, 2.0])
X_test['NetWorthCategory'] = X_test['NetWorthCategory'].astype(float) 
X_test['HighRisk'] = X_test['CreditScore'].apply(lambda x: 1 if x < 600 else 0)#
X_test['IncomeCategory'] = pd.cut(X_test['AnnualIncome'], bins=[0, 30000, 60000, 90000, float('inf')], labels=[1.0, 2.0, 3.0, 4.0])
income_group_mean = X_test.groupby('IncomeCategory')['CreditScore'].transform('mean')
X_test['CreditScoreDeviationIncome'] = X_test['CreditScore'] - income_group_mean
X_test['DebtToAssetRatio'] = X_test['TotalLiabilities'] / X_test['TotalAssets']
X_test['MonthlyLoanPaymentToIncomeRatio'] = X_test['MonthlyLoanPayment'] / X_test['MonthlyIncome']
X_test['UtilityPaymentsToIncomeRatio'] = X_test['UtilityBillsPaymentHistory'] / X_test['MonthlyIncome']
X_test['EmploymentStability'] = X_test['JobTenure'].apply(lambda x: 1 if x >= 5 else 0)
X_test['CreditScoreCategory'] = pd.cut(X_test['CreditScore'], bins=[0, 300, 550, 722, 850, float('inf')], labels=[1.0, 2.0, 3.0, 4.0, 5.0])
X_test['HighCreditScoreFlag'] = X_test['CreditScore'].apply(lambda x: 1 if x >= 740 else 0)#
X_test['DefaultRiskScore'] = X_test['CreditScore'].apply(lambda x: 1 if x < 580 else 0)#
X_test['DefaultRiskScoren'] = X_test['CreditScore'].apply(lambda x: 1 if x < 550 else 0)
X_test['WeightedCreditScore'] = X_test['CreditScore'] * X_test['AnnualIncome']
X_test['IncomeToDebtRatio'] = X_test['AnnualIncome'] / X_test['TotalLiabilities']
hours_per_year = 2080 
X_test['HourlyIncome'] = X_test['AnnualIncome'] / hours_per_year
X_test['IncomeByAgeGroup'] = X_test.groupby('Age')['AnnualIncome'].transform('mean')
X_test['IncomeToBaseInterestRate'] = X_test['AnnualIncome'] / X_test['BaseInterestRate']
X_test['LoanApprovedFlag'] = (X_test['LoanApproved'] == 1).astype(float)
median_loan_amount = X_test['LoanAmount'].median()
X_test['HighLoanAmountFlag'] = (X_test['LoanAmount'] > median_loan_amount).astype(float)
X_test['HighLoanAmountApproval'] = X_test['LoanApproved'] * X_test['HighLoanAmountFlag']
X_test['IncomeToLoanApproval'] = X_test['AnnualIncome'] / (X_test['LoanApproved'] + 1)
X_test['MonthlyPaymentToApproval'] = X_test['MonthlyLoanPayment'] / (X_test['LoanApproved'] + 1)
X_test['DebtToIncomeCategory'] = pd.cut(X_test['DebtToIncomeRatio'], bins=[-float('inf'), 0.2, 0.3, 0.4, 0.499, float('inf')], labels=[0.0, 1.0, 2.0, 3.0, 4.0]) 
X_test['SavingsBalanceCategory'] = pd.cut(X_test['SavingsAccountBalance'], bins=[-float('inf'), 1000, 4000, 8000, float('inf')], labels=[0.0, 1.0, 2.0, 3.0]) 
X_test['LoanAmountCategory'] = pd.cut(X_test['LoanAmount'], bins=[0, 10000, 30000, 50000, 70000, float('inf')], labels=[0.0, 1.0, 2.0, 3.0, 4.0]) 
X_test['TotalAssetsCategory'] = pd.cut(X_test['TotalAssets'], bins=[0, 50000, 150000, 300000, float('inf')], labels=[0.0, 1.0, 2.0, 3.0]) 
X_test['CreditHistoryCategory'] = pd.cut(X_test['LengthOfCreditHistory'], bins=[0, 2, 5, 9, float('inf')], labels=[0.0, 1.0, 2.0, 3.0]) 
X_test['CreditCardUtilizationCategory'] = pd.cut(X_test['CreditCardUtilizationRate'], bins=[-float('inf'), 0.2, 0.3, 0.4, 0.5, float('inf')], labels=[0.0, 1.0, 2.0, 3.0, 4.0])
X_test['PaymentHistoryCategory'] = pd.cut(X_test['PaymentHistory'], bins=[-1, 14, float('inf')], labels=[0.0, 1.0])
X_test['LowDebtToIncomeFlag'] = (X_test['DebtToIncomeCategory'] <= 1.0).astype(float)
X_test['InterestRateToIncomeRatio'] = X_test['InterestRate'] / X_test['AnnualIncome']
X_test['InterestRateToIncomeCategory'] = pd.cut(X_test['InterestRateToIncomeRatio'], bins=[-float('inf'), 1e-7, 2e-7, 5e-6, 1e-5, float('inf')], labels=[0.0, 1.0, 2.0, 3.0, 4.0])
X_test['CheckingBalanceCategory'] = pd.cut(X_test['CheckingAccountBalance'], bins=[-float('inf'), 19000, 35000, 60000, float('inf')], labels=[0.0, 1.0, 2.0, 3.0])

train_df.describe()

/tmp/ipykernel_17/3479759578.py:12: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  income_group_mean = train_df.groupby('IncomeCategory')['CreditScore'].transform('mean')
/tmp/ipykernel_17/3479759578.py:140: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  income_group_mean = X_test.groupby('IncomeCategory')['CreditScore'].transform('mean')


,ApplicationDate,Age,AnnualIncome,CreditScore,LoanAmount,LoanDuration,NumberOfDependents,MonthlyDebtPayments,CreditCardUtilizationRate,NumberOfOpenCreditLines,NumberOfCreditInquiries,DebtToIncomeRatio,BankruptcyHistory,PreviousLoanDefaults,PaymentHistory,LengthOfCreditHistory,SavingsAccountBalance,CheckingAccountBalance,TotalAssets,TotalLiabilities,MonthlyIncome,UtilityBillsPaymentHistory,JobTenure,Experience,NetWorth,BaseInterestRate,InterestRate,MonthlyLoanPayment,TotalDebtToIncomeRatio,LoanApproved,RiskScore,ApprovalRate_HomeOwnershipStatus_Other,ApprovalRate_HomeOwnershipStatus_Own,ApprovalRate_HomeOwnershipStatus_Rent,AssetsToLoanApproval,ApprovalRate_LoanPurpose_Debt Consolidation,ApprovalRate_LoanPurpose_Education,ApprovalRate_LoanPurpose_Home,ApprovalRate_LoanPurpose_Other,ApprovalRate_EducationLevel_Bachelor,ApprovalRate_EducationLevel_Doctorate,ApprovalRate_EducationLevel_High School,ApprovalRate_EducationLevel_Master,ApprovedLoanAmount,HighLoanAmountApproved,GoodCreditRejected,HighIncomeRejected,LowDebtRejected,LongCreditHistoryRejected,HomeOwnerRejected,HighSavingsRejected,HighAssetsRejected,HighNetWorthRejected,LowCreditUtilizationRejected,NoBankruptcyRejected,GoodPaymentHistoryRejected,HighMonthlyIncomeRejected,NoPreviousDefaultsRejected,LowDebtToIncomeRatioRejected,GoodUtilityBillsPaymentHistoryRejected,StableEmploymentRejected,NoCreditDefaultsRejected,LowMonthlyDebtRejected,GoodEducationRejected,HighCheckingBalanceRejected,HighTotalAssetsRejected,GoodCreditHistoryRejected,HighLoanAmountRejected,HighCreditApproved,LowDebtToIncomeRatio,SavingsToIncomeRatio,new,neww,LiquidAssets,newww,LiquidAssetsToLiabilitiesRatio,NetWorthToLiabilitiesRatio,NetWorthCategory,HighRisk,CreditScoreDeviationIncome,DebtToAssetRatio,MonthlyLoanPaymentToIncomeRatio,UtilityPaymentsToIncomeRatio,EmploymentStability,HighCreditScoreFlag,DefaultRiskScore,DefaultRiskScoren,WeightedCreditScore,IncomeToDebtRatio,HourlyIncome,IncomeByAgeGroup,IncomeToBaseInterestRate,LoanApprovedFlag,HighLoanAmountFlag,HighLoanAmountApproval,IncomeToLoanApproval,MonthlyPaymentToApproval,LowDebtToIncomeFlag,InterestRateToIncomeRatio
count,9787.000000,9787.000000,9787.000000,9787.000000,9787.000000,9787.000000,9787.000000,9787.000000,9787.000000,9787.000000,9787.000000,9787.000000,9787.000000,9787.000000,9787.000000,9787.000000,9787.000000,9787.000000,9.787000e+03,9.787000e+03,9787.000000,9787.000000,9787.000000,9787.000000,9.787000e+03,9787.000000,9787.000000,9787.000000,9787.000000,9787.000000,9787.000000,9787.000000,9787.000000,9787.000000,9.787000e+03,9787.000000,9787.000000,9787.000000,9787.000000,9787.000000,9787.000000,9787.000000,9787.000000,9787.000000,9787.000000,9787.000000,9787.000000,9787.000000,9787.000000,9787.000000,9787.000000,9787.000000,9787.000000,9787.000000,9787.000000,9787.0,9787.000000,9787.000000,9787.000000,9787.0,9787.000000,9787.000000,9787.000000,9787.000000,9787.000000,9787.000000,9787.000000,9787.000000,9787.000000,9787.000000,9787.000000,9787.000000,9787.000000,9787.000000,9.787000e+03,9787.000000,9787.000000,9787.000000,9787.000000,9.787000e+03,9787.000000,9787.000000,9787.000000,9787.000000,9787.000000,9787.000000,9787.000000,9.787000e+03,9787.000000,9787.000000,9787.000000,9.787000e+03,9787.000000,9787.000000,9787.000000,9787.000000,9787.000000,9787.000000,9.787000e+03
mean,1998.947686,39.898743,131527.784510,677.911515,29925.122509,53.439052,1.568714,546.437928,0.284935,3.029733,0.976704,0.249999,0.052621,0.096250,24.011035,15.615306,5512.410749,1987.982323,1.823864e+05,6.875512e+04,10831.569727,0.784073,4.938694,17.671299,1.536230e+05,0.200502,0.200155,1079.984544,0.519815,0.512108,48.349800,0.512108,0.512108,0.512108,1.206228e+05,0.512108,0.512108,0.512108,0.512108,0.512108,0.512108,0.512108,0.512108,12607.798202,0.218147,0.167569,0.071115,0.328906,0.420456,0.096352,0.022683,0.144069,0.118525,0.168284,0.457546,0.0,0.043731,0.436395,0.208440,0.0,0.423214,0.436395,0.332278,0.043118,0.057321,0.031675,0.186574,0.449678,0.302851,0.6

In [15]:

train_df['LoanPaymentToLoanAmountRatio'] = train_df['MonthlyLoanPayment'] / train_df['LoanAmount']
train_df['LoanPaymentToDebtToIncomeRatio'] = train_df['MonthlyLoanPayment'] / train_df['TotalDebtToIncomeRatio']
train_df['LoanPaymentAsPercentOfNetWorth'] = train_df['MonthlyLoanPayment'] / train_df['NetWorth']
train_df['LoanPaymentToMonthlyDebtRatio'] = train_df['MonthlyLoanPayment'] / train_df['MonthlyDebtPayments']
train_df['LoanPaymentAsPercentOfTotalDebt'] = train_df['MonthlyLoanPayment'] / train_df['TotalLiabilities']
train_df['HighInterestRateFlag'] = (train_df['InterestRate'] > 0.1).astype(float)
train_df['InterestRateToIncomeRatio'] = train_df['InterestRate'] / train_df['AnnualIncome']
train_df['InterestRateToDebtToIncomeRatio'] = train_df['InterestRate'] / train_df['TotalDebtToIncomeRatio']
mean_rate = train_df['InterestRate'].mean()
std_rate = train_df['InterestRate'].std()
train_df['InterestRateFluctuationFlag'] = ((train_df['InterestRate'] > (mean_rate + 2 * std_rate)) | (train_df['InterestRate'] < (mean_rate - 2 * std_rate))).astype(float)
train_df['InterestRateToCreditCardUtilization'] = train_df['InterestRate'] / (train_df['CreditCardUtilizationRate'] + 1)
train_df['InterestRateForFirstLoanFlag'] = ((train_df['PreviousLoanDefaults'] == 0) & (train_df['InterestRate'] > 0.10)).astype(float)
train_df['HighInterestForDebtConsolidation'] = ((train_df['LoanPurpose_Debt Consolidation'] == True) & (train_df['InterestRate'] > 0.06)).astype(float)




X_test['LoanPaymentToLoanAmountRatio'] = X_test['MonthlyLoanPayment'] / X_test['LoanAmount']
X_test['LoanPaymentToDebtToIncomeRatio'] = X_test['MonthlyLoanPayment'] / X_test['TotalDebtToIncomeRatio']
X_test['LoanPaymentAsPercentOfNetWorth'] = X_test['MonthlyLoanPayment'] / X_test['NetWorth']
X_test['LoanPaymentToMonthlyDebtRatio'] = X_test['MonthlyLoanPayment'] / X_test['MonthlyDebtPayments']
X_test['LoanPaymentAsPercentOfTotalDebt'] = X_test['MonthlyLoanPayment'] / X_test['TotalLiabilities']
X_test['HighInterestRateFlag'] = (X_test['InterestRate'] > 0.1).astype(float)
X_test['InterestRateToIncomeRatio'] = X_test['InterestRate'] / X_test['AnnualIncome']
X_test['InterestRateToDebtToIncomeRatio'] = X_test['InterestRate'] / X_test['TotalDebtToIncomeRatio']
mean_rate = X_test['InterestRate'].mean()
std_rate = X_test['InterestRate'].std()
X_test['InterestRateFluctuationFlag'] = ((X_test['InterestRate'] > (mean_rate + 2 * std_rate)) | (X_test['InterestRate'] < (mean_rate - 2 * std_rate))).astype(float)
X_test['InterestRateToCreditCardUtilization'] = X_test['InterestRate'] / (X_test['CreditCardUtilizationRate'] + 1)
X_test['InterestRateForFirstLoanFlag'] = ((X_test['PreviousLoanDefaults'] == 0) & (X_test['InterestRate'] > 0.10)).astype(float)
X_test['HighInterestForDebtConsolidation'] = ((X_test['LoanPurpose_Debt Consolidation'] == True) & (X_test['InterestRate'] > 0.06)).astype(float)

In [16]:
train_df['LowIncomeRisk'] = train_df['AnnualIncome'].apply(lambda x: 1 if x < 36000 else 0)
train_df['LongLoanDurationRisk'] = train_df['LoanDuration'].apply(lambda x: 1 if x > 35 else 0)
train_df['MultipleCreditInquiriesRisk'] = train_df['NumberOfCreditInquiries'].apply(lambda x: 1 if x > 4 else 0)
train_df['LowAssetsRisk'] = train_df['TotalAssets'].apply(lambda x: 1 if x < 5000 else 0)
train_df['ShortLoanDurationRisk'] = train_df['LoanDuration'].apply(lambda x: 1 if x < 18 else 0)
train_df['ManyOpenCreditLinesRisk'] = train_df['NumberOfOpenCreditLines'].apply(lambda x: 1 if x > 9 else 0)
train_df['LateUtilityPaymentPenalty'] = 1 - train_df['UtilityBillsPaymentHistory']
train_df['LowCreditAndPoorUtilityPayment'] = train_df.apply(lambda x: 1 if (x['CreditScore'] < 600) and (x['UtilityBillsPaymentHistory'] < 0.3) else 0, axis=1)





X_test['LowIncomeRisk'] = X_test['AnnualIncome'].apply(lambda x: 1 if x < 36000 else 0)
X_test['LongLoanDurationRisk'] = X_test['LoanDuration'].apply(lambda x: 1 if x > 35 else 0)
X_test['MultipleCreditInquiriesRisk'] = X_test['NumberOfCreditInquiries'].apply(lambda x: 1 if x > 4 else 0)
X_test['LowAssetsRisk'] = X_test['TotalAssets'].apply(lambda x: 1 if x < 5000 else 0)
X_test['ShortLoanDurationRisk'] = X_test['LoanDuration'].apply(lambda x: 1 if x < 18 else 0)
X_test['ManyOpenCreditLinesRisk'] = X_test['NumberOfOpenCreditLines'].apply(lambda x: 1 if x > 9 else 0)
X_test['LateUtilityPaymentPenalty'] = 1 - X_test['UtilityBillsPaymentHistory']
X_test['LowCreditAndPoorUtilityPayment'] = X_test.apply(lambda x: 1 if (x['CreditScore'] < 600) and (x['UtilityBillsPaymentHistory'] < 0.3) else 0, axis=1)

In [17]:
train_df['CheckingAccountBalanceСat'] = pd.cut(train_df['CheckingAccountBalance'], bins=[-float('inf'), 800, 2500, 4900, float('inf')], labels=[0.0, 1.0, 2.0, 3.0]) 
train_df['CreditUtilizationCategory'] = pd.cut(train_df['CreditCardUtilizationRate'], bins=[-float('inf'), 0.2, 0.4, 0.6, float('inf')], labels=[0.0, 1.0, 2.0, 3.0])
train_df['HighCreditUtilizationFlag'] = train_df['CreditCardUtilizationRate'].apply(lambda x: 1 if x > 0.5 else 0)
train_df['CreditUtilizationDebtInteraction'] = train_df['CreditCardUtilizationRate'] * train_df['DebtToIncomeRatio']
train_df['ExtremeDebtToIncomeFlag'] =  train_df['DebtToIncomeRatio'].apply(lambda x: 1 if x < 0.4 or x > 0.6 else 0)
train_df['DebtToIncomeCreditRiskFlag'] = ((train_df['DebtToIncomeRatio'] > 0.35) & (train_df['LoanApproved'] == 0)).astype(float)
train_df['DebtToCreditLinesRatio'] = train_df['DebtToIncomeRatio'] / train_df['NumberOfOpenCreditLines']
train_df['DebtToCreditLinesRatio'] = train_df['DebtToCreditLinesRatio'].replace([np.inf, -np.inf], np.nan)
train_df['DebtToCreditLinesRatio'] = train_df['DebtToCreditLinesRatio'].fillna(0)  
train_df['DebtToCreditLinesRatio'] = np.clip(train_df['DebtToCreditLinesRatio'], -1e6, 1e6) 
train_df['ProgressiveDebtToIncomeFlag'] = (train_df['DebtToIncomeRatio'] > train_df['DebtToIncomeRatio'].shift(1)).astype(float)
train_df['DebtToIncomeCreditInquiriesInteraction'] = train_df['DebtToIncomeRatio'] * train_df['NumberOfCreditInquiries']
train_df['HighDebtLowIncomeFlag'] = ((train_df['DebtToIncomeRatio'] > 0.4) & (train_df['MonthlyIncome'] < 1000)).astype(float)
train_df['HighDebtLowSavingsFlag'] = ((train_df['DebtToIncomeRatio'] > 0.4) & (train_df['SavingsAccountBalance'] < 485)).astype(float)
train_df['LowDebtHighIncomeFlag'] = ((train_df['DebtToIncomeRatio'] < 0.4) & (train_df['MonthlyIncome'] > 8100)).astype(float)
train_df['HighInterestLowCreditScoreFlag'] = ((train_df['InterestRate'] > 0.1) & (train_df['CreditScore'] < 650)).astype(float)
train_df['HighInterestLowIncomeFlag'] = ((train_df['InterestRate'] > 0.1) & (train_df['MonthlyIncome'] < 2000)).astype(float)
train_df['HighInterestHighLiabilitiesFlag'] = ((train_df['InterestRate'] > 0.1) & (train_df['TotalLiabilities'] > 30000)).astype(float)
train_df['HighCreditUtilizationLowCreditScoreFlag'] = ((train_df['CreditCardUtilizationRate'] > 0.6) & (train_df['CreditScore'] < 650)).astype(float)
train_df['HighInterestLowAssetsFlag'] = ((train_df['InterestRate'] > 0.1) & (train_df['TotalAssets'] < 15000)).astype(float)
train_df['LowIncomeHighDebtFlag'] = ((train_df['MonthlyIncome'] < 1000) & (train_df['DebtToIncomeRatio'] > 0.5)).astype(float)
train_df['LowIncomeHighLoanAmountFlag'] = ((train_df['MonthlyIncome'] < 2500) & (train_df['LoanAmount'] > 40000)).astype(float)
train_df['LowCreditScoreHighLoanAmountFlag'] = ((train_df['CreditScore'] < 650) & (train_df['LoanAmount'] > 19900)).astype(float)
train_df['LongLoanDurationLowIncomeFlag'] = ((train_df['LoanDuration'] > 50) & (train_df['MonthlyIncome'] < 1500)).astype(float)
train_df['HighCreditUtilizationLowIncomeFlag'] = ((train_df['CreditCardUtilizationRate'] > 0.7) & (train_df['MonthlyIncome'] < 1500)).astype(float)
train_df['LowCreditScoreHighDebtFlag'] = ((train_df['CreditScore'] < 680) & (train_df['DebtToIncomeRatio'] > 0.15)).astype(float)
train_df['LowCreditScoreHighCreditUtilizationFlag'] = ((train_df['CreditScore'] < 600) & (train_df['CreditCardUtilizationRate'] > 0.15)).astype(float)
train_df['LowCreditScoreHighMonthlyDebtFlag'] = ((train_df['CreditScore'] < 600) & (train_df['MonthlyDebtPayments'] > 1500)).astype(float)
train_df['YoungAgeHighLoanAmountFlag'] = ((train_df['Age'] < 25) & (train_df['LoanAmount'] > 60000)).astype(float)
train_df['HighCreditScoreLowJobTenureFlag'] = ((train_df['CreditScore'] > 700) & (train_df['JobTenure'] < 2)).astype(float)
train_df['LowIncomeHighInterestRateFlag'] = ((train_df['MonthlyIncome'] < 4200) & (train_df['InterestRate'] > 0.1)).astype(float)






X_test['CheckingAccountBalanceСat'] = pd.cut(X_test['CheckingAccountBalance'], bins=[-float('inf'), 800, 2500, 4900, float('inf')], labels=[0.0, 1.0, 2.0, 3.0])
X_test['CreditUtilizationCategory'] = pd.cut(X_test['CreditCardUtilizationRate'], bins=[-float('inf'), 0.2, 0.4, 0.6, float('inf')], labels=[0.0, 1.0, 2.0, 3.0])
X_test['HighCreditUtilizationFlag'] = X_test['CreditCardUtilizationRate'].apply(lambda x: 1 if x > 0.55 else 0)
X_test['CreditUtilizationDebtInteraction'] = X_test['CreditCardUtilizationRate'] * X_test['DebtToIncomeRatio']
X_test['ExtremeDebtToIncomeFlag'] = X_test['DebtToIncomeRatio'].apply(lambda x: 1 if x < 0.4 or x > 0.6 else 0)
X_test['DebtToIncomeCreditRiskFlag'] = ((X_test['DebtToIncomeRatio'] > 0.35) & (X_test['LoanApproved'] == 0)).astype(float)
X_test['DebtToCreditLinesRatio'] = X_test['DebtToIncomeRatio'] / X_test['NumberOfOpenCreditLines']
X_test['DebtToCreditLinesRatio'] = X_test['DebtToCreditLinesRatio'].replace([np.inf, -np.inf], np.nan)
X_test['DebtToCreditLinesRatio'] = X_test['DebtToCreditLinesRatio'].fillna(0)  
X_test['DebtToCreditLinesRatio'] = np.clip(X_test['DebtToCreditLinesRatio'], -1e6, 1e6)
X_test['ProgressiveDebtToIncomeFlag'] = (X_test['DebtToIncomeRatio'] > X_test['DebtToIncomeRatio'].shift(1)).astype(float)
X_test['DebtToIncomeCreditInquiriesInteraction'] = X_test['DebtToIncomeRatio'] * X_test['NumberOfCreditInquiries']
X_test['HighDebtLowIncomeFlag'] = ((X_test['DebtToIncomeRatio'] > 0.4) & (X_test['MonthlyIncome'] < 1000)).astype(float)
X_test['HighDebtLowSavingsFlag'] = ((X_test['DebtToIncomeRatio'] > 0.4) & (X_test['SavingsAccountBalance'] < 485)).astype(float)
X_test['LowDebtHighIncomeFlag'] = ((X_test['DebtToIncomeRatio'] < 0.4) & (X_test['MonthlyIncome'] > 8100)).astype(float)
X_test['HighInterestLowCreditScoreFlag'] = ((X_test['InterestRate'] > 0.1) & (X_test['CreditScore'] < 650)).astype(float)
X_test['HighInterestLowIncomeFlag'] = ((X_test['InterestRate'] > 0.1) & (X_test['MonthlyIncome'] < 2000)).astype(float)
X_test['HighInterestHighLiabilitiesFlag'] = ((X_test['InterestRate'] > 0.1) & (X_test['TotalLiabilities'] > 30000)).astype(float)
X_test['HighCreditUtilizationLowCreditScoreFlag'] = ((X_test['CreditCardUtilizationRate'] > 0.6) & (X_test['CreditScore'] < 650)).astype(float)
X_test['HighInterestLowAssetsFlag'] = ((X_test['InterestRate'] > 0.1) & (X_test['TotalAssets'] < 15000)).astype(float)
X_test['LowIncomeHighDebtFlag'] = ((X_test['MonthlyIncome'] < 1000) & (X_test['DebtToIncomeRatio'] > 0.5)).astype(float)
X_test['LowIncomeHighLoanAmountFlag'] = ((X_test['MonthlyIncome'] < 2500) & (X_test['LoanAmount'] > 40000)).astype(float)
X_test['LowCreditScoreHighLoanAmountFlag'] = ((X_test['CreditScore'] < 650) & (X_test['LoanAmount'] > 19900)).astype(float)
X_test['LongLoanDurationLowIncomeFlag'] = ((X_test['LoanDuration'] > 50) & (X_test['MonthlyIncome'] < 1500)).astype(float)
X_test['HighCreditUtilizationLowIncomeFlag'] = ((X_test['CreditCardUtilizationRate'] > 0.7) & (X_test['MonthlyIncome'] < 1500)).astype(float)
X_test['LowCreditScoreHighDebtFlag'] = ((X_test['CreditScore'] < 680) & (X_test['DebtToIncomeRatio'] > 0.15)).astype(float)
X_test['LowCreditScoreHighCreditUtilizationFlag'] = ((X_test['CreditScore'] < 600) & (X_test['CreditCardUtilizationRate'] > 0.15)).astype(float)
X_test['LowCreditScoreHighMonthlyDebtFlag'] = ((X_test['CreditScore'] < 600) & (X_test['MonthlyDebtPayments'] > 1500)).astype(float)
X_test['YoungAgeHighLoanAmountFlag'] = ((X_test['Age'] < 25) & (X_test['LoanAmount'] > 60000)).astype(float)
X_test['HighCreditScoreLowJobTenureFlag'] = ((X_test['CreditScore'] > 700) & (X_test['JobTenure'] < 2)).astype(float)
X_test['LowIncomeHighInterestRateFlag'] = ((X_test['MonthlyIncome'] < 4200) & (X_test['InterestRate'] > 0.1)).astype(float)


/tmp/ipykernel_17/1281371488.py:49: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  train_df['LowCreditScoreHighLoanAmountFlag'] = ((train_df['CreditScore'] < 650) & (train_df['LoanAmount'] > 19900)).astype(float)
/tmp/ipykernel_17/1281371488.py:51: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  train_df['LongLoanDurationLowIncomeFlag'] = ((train_df['LoanDuration'] > 50) & (train_df['MonthlyIncome'] < 1500)).astype(float)
/tmp/ipykernel_17/1281371488.py:53: PerformanceWarning: DataFrame is highly fragmented.  This is usually the re

In [19]:
train_df = train_df.reset_index(drop=True)

y_bobr = train_df['RiskScore']
X_bobr = train_df.drop(columns=['RiskScore'])

In [20]:
#4.деление на тренеровочную и валидациооную
X_train, X_valid, y_train, y_valid = train_test_split(
    X_bobr, y_bobr, test_size=0.2, random_state=0)

X_train = X_train.reset_index(drop=True)
X_valid = X_valid.reset_index(drop=True)

In [21]:

from sklearn.preprocessing import PolynomialFeatures


#train
features_to_transform = X_train[['CreditScore']]
poly = PolynomialFeatures(degree=52, include_bias=False)#10 11 12 13 14 не апроверял 
poly_features = poly.fit_transform(features_to_transform)
poly_features_df = pd.DataFrame(poly_features, columns=poly.get_feature_names_out(features_to_transform.columns))
X_train = pd.concat([X_train, poly_features_df], axis=1)

#valid
features_to_transform = X_valid[['CreditScore']]
poly_features = poly.transform(features_to_transform)
poly_features_df = pd.DataFrame(poly_features, columns=poly.get_feature_names_out(features_to_transform.columns))
X_valid = pd.concat([X_valid, poly_features_df], axis=1)
#test
features_to_transform = X_test[['CreditScore']]
poly_features = poly.transform(features_to_transform)
poly_features_df = pd.DataFrame(poly_features, columns=poly.get_feature_names_out(features_to_transform.columns))
X_test = pd.concat([X_test, poly_features_df], axis=1)



#train
features_to_transform = X_train[['MonthlyIncome']]
poly = PolynomialFeatures(degree=3, include_bias=False)
poly_features = poly.fit_transform(features_to_transform)
poly_features_df = pd.DataFrame(poly_features, columns=poly.get_feature_names_out(features_to_transform.columns))
X_train = pd.concat([X_train, poly_features_df], axis=1)
#valid
features_to_transform = X_valid[['MonthlyIncome']]
poly_features = poly.transform(features_to_transform)
poly_features_df = pd.DataFrame(poly_features, columns=poly.get_feature_names_out(features_to_transform.columns))
X_valid = pd.concat([X_valid, poly_features_df], axis=1)
#test
features_to_transform = X_test[['MonthlyIncome']]
poly_features = poly.transform(features_to_transform)
poly_features_df = pd.DataFrame(poly_features, columns=poly.get_feature_names_out(features_to_transform.columns))
X_test = pd.concat([X_test, poly_features_df], axis=1)

# Удаление дубликатов столбцов
X_train = X_train.loc[:, ~X_train.columns.duplicated()]
X_valid = X_valid.loc[:, ~X_valid.columns.duplicated()]
X_test = X_test.loc[:, ~X_test.columns.duplicated()]
X_train

,ApplicationDate,Age,AnnualIncome,CreditScore,LoanAmount,LoanDuration,NumberOfDependents,MonthlyDebtPayments,CreditCardUtilizationRate,NumberOfOpenCreditLines,NumberOfCreditInquiries,DebtToIncomeRatio,BankruptcyHistory,PreviousLoanDefaults,PaymentHistory,LengthOfCreditHistory,SavingsAccountBalance,CheckingAccountBalance,TotalAssets,TotalLiabilities,MonthlyIncome,UtilityBillsPaymentHistory,JobTenure,Experience,NetWorth,BaseInterestRate,InterestRate,MonthlyLoanPayment,TotalDebtToIncomeRatio,LoanApproved,MaritalStatus_TargetEncoded,Mar_Divorced,Mar_Married,Mar_Single,Emp_Employed,Emp_Self-Employed,HomeOwnershipStatus_Other,HomeOwnershipStatus_Own,HomeOwnershipStatus_Rent,ApprovalRate_HomeOwnershipStatus_Other,ApprovalRate_HomeOwnershipStatus_Own,ApprovalRate_HomeOwnershipStatus_Rent,AssetsToLoanApproval,LoanPurpose_Debt Consolidation,LoanPurpose_Education,LoanPurpose_Home,LoanPurpose_Other,ApprovalRate_LoanPurpose_Debt Consolidation,ApprovalRate_LoanPurpose_Education,ApprovalRate_LoanPurpose_Home,ApprovalRate_LoanPurpose_Other,EducationLevel_Bachelor,EducationLevel_Doctorate,EducationLevel_High School,EducationLevel_Master,ApprovalRate_EducationLevel_Bachelor,ApprovalRate_EducationLevel_Doctorate,ApprovalRate_EducationLevel_High School,ApprovalRate_EducationLevel_Master,ApprovedLoanAmount,HighLoanAmountApproved,GoodCreditRejected,HighIncomeRejected,LowDebtRejected,LongCreditHistoryRejected,HomeOwnerRejected,HighSavingsRejected,HighAssetsRejected,HighNetWorthRejected,LowCreditUtilizationRejected,NoBankruptcyRejected,GoodPaymentHistoryRejected,HighMonthlyIncomeRejected,NoPreviousDefaultsRejected,LowDebtToIncomeRatioRejected,GoodUtilityBillsPaymentHistoryRejected,StableEmploymentRejected,NoCreditDefaultsRejected,LowMonthlyDebtRejected,GoodEducationRejected,HighCheckingBalanceRejected,HighTotalAssetsRejected,GoodCreditHistoryRejected,HighLoanAmountRejected,HighCreditApproved,LowDebtToIncomeRatio,SavingsToIncomeRatio,new,neww,LiquidAssets,newww,LiquidAssetsToLiabilitiesRatio,NetWorthToLiabilitiesRatio,NetWorthCategory,HighRisk,IncomeCategory,CreditScoreDeviationIncome,DebtToAssetRatio,MonthlyLoanPaymentToIncomeRatio,UtilityPaymentsToIncomeRatio,EmploymentStability,CreditScoreCategory,HighCreditScoreFlag,DefaultRiskScore,DefaultRiskScoren,WeightedCreditScore,IncomeToDebtRatio,HourlyIncome,IncomeByAgeGroup,IncomeToBaseInterestRate,LoanApprovedFlag,HighLoanAmountFlag,HighLoanAmountApproval,IncomeToLoanApproval,MonthlyPaymentToApproval,DebtToIncomeCategory,SavingsBalanceCategory,LoanAmountCategory,TotalAssetsCategory,CreditHistoryCategory,CreditCardUtilizationCategory,PaymentHistoryCategory,LowDebtToIncomeFlag,InterestRateToIncomeRatio,InterestRateToIncomeCategory,CheckingBalanceCategory,LoanPaymentToLoanAmountRatio,LoanPaymentToDebtToIncomeRatio,LoanPaymentAsPercentOfNetWorth,LoanPaymentToMonthlyDebtRatio,LoanPaymentAsPercentOfTotalDebt,HighInterestRateFlag,InterestRateToDebtToIncomeRatio,InterestRateFluctuationFlag,InterestRateToCreditCardUtilization,InterestRateForFirstLoanFlag,HighInterestForDebtConsolidation,LowIncomeRisk,LongLoanDurationRisk,MultipleCreditInquiriesRisk,LowAssetsRisk,ShortLoanDurationRisk,ManyOpenCreditLinesRisk,LateUtilityPaymentPenalty,LowCreditAndPoorUtilityPayment,CheckingAccountBalanceСat,CreditUtilizationCategory,HighCreditUtilizationFlag,CreditUtilizationDebtInteraction,ExtremeDebtToIncomeFlag,DebtToIncomeCreditRiskFlag,DebtToCreditLinesRatio,ProgressiveDebtToIncomeFlag,DebtToIncomeCreditInquiriesInteraction,HighDebtLowIncomeFlag,HighDebtLowSavingsFlag,LowDebtHighIncomeFlag,HighInterestLowCreditScoreFlag,HighInterestLowIncomeFlag,HighInterestHighLiabilitiesFlag,HighCreditUtilizationLowCreditScoreFlag,HighInterestLowAssetsFlag,LowIncomeHighDebtFlag,LowIncomeHighLoanAmountFlag,LowCreditScoreHighLoanAmountFlag,LongLoanDurationLowIncomeFlag,HighCreditUtilizationLowIncomeFlag,LowCreditScoreHighDebtFlag,LowCreditScoreHighCreditUtilizationFlag,LowCreditScoreHighMonthlyDebtFlag,YoungAgeHighLoanAmountFlag,HighCreditSco

In [22]:

name_c3 = 'TotalAssets'
name_cc3 = 'TotalAssetsnew'
X_train[name_cc3] = (X_train[name_c3] + 1) ** (1/12) 
X_valid[name_cc3] = (X_valid[name_c3] + 1) ** (1/12)
X_test[name_cc3] = (X_test[name_c3] + 1) ** (1/12)
X_train = X_train.drop(columns=[name_c3])
X_valid = X_valid.drop(columns=[name_c3])
X_test = X_test.drop(columns=[name_c3])

name_c7 = 'TotalLiabilities'
name_cc7 = 'TotalLiabilitiesnew'
X_train[name_cc7] = np.log(X_train[name_c7] + 1) 
X_valid[name_cc7] = np.log(X_valid[name_c7] + 1)
X_test[name_cc7] = np.log(X_test[name_c7] + 1)
X_train = X_train.drop(columns=[name_c7])
X_valid = X_valid.drop(columns=[name_c7])
X_test = X_test.drop(columns=[name_c7])

name_c8 = 'NetWorth'
name_cc8 = 'NetWorthnew'
X_train[name_cc8] = (X_train[name_c8] + 1) ** (1/6) 
X_valid[name_cc8] = (X_valid[name_c8] + 1) ** (1/6)
X_test[name_cc8] = (X_test[name_c8] + 1) ** (1/6) 
X_train = X_train.drop(columns=[name_c8])
X_valid = X_valid.drop(columns=[name_c8])
X_test = X_test.drop(columns=[name_c8])

name_c5 = 'MonthlyLoanPayment'
name_cc5 = 'MonthlyLoanPaymentnew'
X_train[name_cc5] = np.log(X_train[name_c5] + 1)  
X_valid[name_cc5] = np.log(X_valid[name_c5] + 1)
X_test[name_cc5] = np.log(X_test[name_c5] + 1)
X_train = X_train.drop(columns=[name_c5])
X_valid = X_valid.drop(columns=[name_c5])
X_test = X_test.drop(columns=[name_c5])

name_c6 = 'TotalDebtToIncomeRatio'
name_cc6 = 'TotalDebtToIncomeRationew'
X_train[name_cc6] = np.sqrt(X_train[name_c6] + 1)  
X_valid[name_cc6] = np.sqrt(X_valid[name_c6] + 1)
X_test[name_cc6] = np.sqrt(X_test[name_c6] + 1)
X_train = X_train.drop(columns=[name_c6])
X_valid = X_valid.drop(columns=[name_c6])
X_test = X_test.drop(columns=[name_c6])

name_c1 = 'LoanAmount'
name_cc1 = 'LoanAmountnew'
X_train[name_cc1] = (X_train[name_c1]) ** (1/3) 
X_valid[name_cc1] = (X_valid[name_c1]) ** (1/3)
X_test[name_cc1] = (X_test[name_c1]) ** (1/3) 
X_train = X_train.drop(columns=[name_c1])
X_valid = X_valid.drop(columns=[name_c1])
X_test = X_test.drop(columns=[name_c1])

name_c2 = 'AnnualIncome'
name_cc2 = 'AnnualIncomenew'
X_train[name_cc2] = (X_train[name_c2]) ** (1/6) 
X_valid[name_cc2] = (X_valid[name_c2]) ** (1/6) 
X_test[name_cc2] = (X_test[name_c2]) ** (1/6) 
X_train = X_train.drop(columns=[name_c2])
X_valid = X_valid.drop(columns=[name_c2])
X_test = X_test.drop(columns=[name_c2])

name_c4 = 'Experience'
name_cc4 = 'Experiencenew'
X_train[name_cc4] = (X_train[name_c4]) ** (1/10) 
X_valid[name_cc4] = (X_valid[name_c4]) ** (1/10) 
X_test[name_cc4] = (X_test[name_c4]) ** (1/10)
X_train = X_train.drop(columns=[name_c4])
X_valid = X_valid.drop(columns=[name_c4])
X_test = X_test.drop(columns=[name_c4])

name_c11 = 'BaseInterestRate'
name_cc11 = 'BaseInterestRatenew'
X_train[name_cc11] = np.sqrt(X_train[name_c11])  
X_valid[name_cc11] = np.sqrt(X_valid[name_c11])
X_test[name_cc11] = np.sqrt(X_test[name_c11])
X_train = X_train.drop(columns=[name_c11])
X_valid = X_valid.drop(columns=[name_c11])
X_test = X_test.drop(columns=[name_c11])

name_c12 = 'InterestRate'
name_cc12 = 'InterestRatenew'
X_train[name_cc12] = np.log(X_train[name_c12] + 1)  
X_valid[name_cc12] = np.log(X_valid[name_c12] + 1) 
X_test[name_cc12] = np.log(X_test[name_c12] + 1)
X_train = X_train.drop(columns=[name_c12])
X_valid = X_valid.drop(columns=[name_c12])
X_test = X_test.drop(columns=[name_c12])

name_c13 = 'LiquidAssets'
name_cc13 = 'LiquidAssetsnew'
X_train[name_cc13] = np.log(X_train[name_c13] + 1)  
X_valid[name_cc13] = np.log(X_valid[name_c13] + 1) 
X_test[name_cc13] = np.log(X_test[name_c13] + 1)
X_train = X_train.drop(columns=[name_c13])
X_valid = X_valid.drop(columns=[name_c13])
X_test = X_test.drop(columns=[name_c13])

name_c14 = 'NetWorthToLiabilitiesRatio'
name_cc14 = 'NetWorthToLiabilitiesRationew'
X_train[name_cc14] = (X_train[name_c14]) ** (1/80)  
X_valid[name_cc14] = (X_valid[name_c14]) ** (1/80)
X_test[name_cc14] = (X_test[name_c14]) ** (1/80)
X_train = X_train.drop(columns=[name_c14])
X_valid = X_valid.drop(columns=[name_c14])
X_test = X_test.drop(columns=[name_c14])

name_c15 = 'DebtToAssetRatio'
name_cc15 = 'DebtToAssetRationew'
X_train[name_cc15] = (X_train[name_c15]) ** (1/6) 
X_valid[name_cc15] = (X_valid[name_c15]) ** (1/6)
X_test[name_cc15] = (X_test[name_c15]) ** (1/6)
X_train = X_train.drop(columns=[name_c15])
X_valid = X_valid.drop(columns=[name_c15])
X_test = X_test.drop(columns=[name_c15])

name_c16 = 'MonthlyLoanPaymentToIncomeRatio'
name_cc16 = 'MonthlyLoanPaymentToIncomeRationew'
X_train[name_cc16] = (X_train[name_c16]) ** (1/20) 
X_valid[name_cc16] = (X_valid[name_c16]) ** (1/20)
X_test[name_cc16] = (X_test[name_c16]) ** (1/20)
X_train = X_train.drop(columns=[name_c16])
X_valid = X_valid.drop(columns=[name_c16])
X_test = X_test.drop(columns=[name_c16])

name_c17 = 'WeightedCreditScore'
name_cc17 = 'WeightedCreditScorenew'
X_train[name_cc17] = (X_train[name_c17]) ** (1/7) 
X_valid[name_cc17] = (X_valid[name_c17]) ** (1/7)
X_test[name_cc17] = (X_test[name_c17]) ** (1/7)
X_train = X_train.drop(columns=[name_c17])
X_valid = X_valid.drop(columns=[name_c17])
X_test = X_test.drop(columns=[name_c17])

name_c18 = 'IncomeToDebtRatio'
name_cc18 = 'IncomeToDebtRationew'
X_train[name_cc18] = (X_train[name_c18]) ** (1/2)  
X_valid[name_cc18] = (X_valid[name_c18]) ** (1/2)
X_test[name_cc18] = (X_test[name_c18]) ** (1/2)
X_train = X_train.drop(columns=[name_c18])
X_valid = X_valid.drop(columns=[name_c18])
X_test = X_test.drop(columns=[name_c18])

name_c19 = 'IncomeToLoanApproval'
name_cc19 = 'IncomeToLoanApprovalnew'
X_train[name_cc19] = (X_train[name_c19]) ** (1/5)  
X_valid[name_cc19] = (X_valid[name_c19]) ** (1/5)
X_test[name_cc19] = (X_test[name_c19]) ** (1/5)
X_train = X_train.drop(columns=[name_c19])
X_valid = X_valid.drop(columns=[name_c19])
X_test = X_test.drop(columns=[name_c19])

name_c20 = 'MonthlyPaymentToApproval'
name_cc20 = 'MonthlyPaymentToApprovalnew'
X_train[name_cc20] = (X_train[name_c20]) ** (1/3)  
X_valid[name_cc20] = (X_valid[name_c20]) ** (1/3)
X_test[name_cc20] = (X_test[name_c20]) ** (1/3)
X_train = X_train.drop(columns=[name_c20])
X_valid = X_valid.drop(columns=[name_c20])
X_test = X_test.drop(columns=[name_c20])

name_c21 = 'AssetsToLoanApproval'
name_cc21 = 'AssetsToLoanApprovalnew'
X_train[name_cc21] = (X_train[name_c21]) ** (1/16)  
X_valid[name_cc21] = (X_valid[name_c21]) ** (1/16)
X_test[name_cc21] = (X_test[name_c21]) ** (1/16)
X_train = X_train.drop(columns=[name_c21])
X_valid = X_valid.drop(columns=[name_c21])
X_test = X_test.drop(columns=[name_c21])

name_c22 = 'ApprovedLoanAmount'
name_cc22 = 'ApprovedLoanAmountnew'
X_train[name_cc22] = (X_train[name_c22]) ** (1/16) 
X_valid[name_cc22] = (X_valid[name_c22]) ** (1/16)
X_test[name_cc22] = (X_test[name_c22]) ** (1/16)
X_train = X_train.drop(columns=[name_c22])
X_valid = X_valid.drop(columns=[name_c22])
X_test = X_test.drop(columns=[name_c22])

name_c23 = 'SavingsToIncomeRatio'
name_cc23 = 'SavingsToIncomeRationew'
X_train[name_cc23] = (X_train[name_c23]) ** (1/2) 
X_valid[name_cc23] = (X_valid[name_c23]) ** (1/2)
X_test[name_cc23] = (X_test[name_c23]) ** (1/2)
X_train = X_train.drop(columns=[name_c23])
X_valid = X_valid.drop(columns=[name_c23])
X_test = X_test.drop(columns=[name_c23])

name_c24 = 'InterestRateToCreditCardUtilization'
name_cc24 = 'InterestRateToCreditCardUtilizationnew'
X_train[name_cc24] = (X_train[name_c24]) ** (1/2) 
X_valid[name_cc24] = (X_valid[name_c24]) ** (1/2)
X_test[name_cc24] = (X_test[name_c24]) ** (1/2)
X_train = X_train.drop(columns=[name_c24])
X_valid = X_valid.drop(columns=[name_c24])
X_test = X_test.drop(columns=[name_c24])

name_c25 = 'InterestRateToDebtToIncomeRatio'
name_cc25 = 'InterestRateToDebtToIncomeRationew'
X_train[name_cc25] = (X_train[name_c25]) ** (1/2)  
X_valid[name_cc25] = (X_valid[name_c25]) ** (1/2)
X_test[name_cc25] = (X_test[name_c25]) ** (1/2)
X_train = X_train.drop(columns=[name_c25])
X_valid = X_valid.drop(columns=[name_c25])
X_test = X_test.drop(columns=[name_c25])

name_c26 = 'CreditScore^2'
name_cc26 = 'CreditScore^2new'
X_train[name_cc26] = (X_train[name_c26]) ** (1/160) 
X_valid[name_cc26] = (X_valid[name_c26]) ** (1/160)
X_test[name_cc26] = (X_test[name_c26]) ** (1/160)
X_train = X_train.drop(columns=[name_c26])
X_valid = X_valid.drop(columns=[name_c26])
X_test = X_test.drop(columns=[name_c26])

name_c27 = 'CreditScore^3'
name_cc27 = 'CreditScore^3new'
X_train[name_cc27] = (X_train[name_c27]) ** (1/8) 
X_valid[name_cc27] = (X_valid[name_c27]) ** (1/8)
X_test[name_cc27] = (X_test[name_c27]) ** (1/8)
X_train = X_train.drop(columns=[name_c27])
X_valid = X_valid.drop(columns=[name_c27])
X_test = X_test.drop(columns=[name_c27])
#
name_c28 = 'CreditScore^5'
name_cc28 = 'CreditScore^5new'
X_train[name_cc28] = (X_train[name_c28]) ** (1/4) 
X_valid[name_cc28] = (X_valid[name_c28]) ** (1/4)
X_test[name_cc28] = (X_test[name_c28]) ** (1/4)
X_train = X_train.drop(columns=[name_c28])
X_valid = X_valid.drop(columns=[name_c28])
X_test = X_test.drop(columns=[name_c28])

name_c29 = 'CreditUtilizationDebtInteraction'
name_cc29 = 'CreditUtilizationDebtInteractionnew'
X_train[name_cc29] = (X_train[name_c29]) ** (1/3)  
X_valid[name_cc29] = (X_valid[name_c29]) ** (1/3)
X_test[name_cc29] = (X_test[name_c29]) ** (1/3)
X_train = X_train.drop(columns=[name_c29])
X_valid = X_valid.drop(columns=[name_c29])
X_test = X_test.drop(columns=[name_c29])


'train_df.info()\n\nimport seaborn as sns\ncorr_matrix = X_train.corr()\nplt.figure(figsize=(35, 29))\nsns.heatmap(corr_matrix, annot=True, fmt=\'.2f\', square=True, cbar_kws={"shrink": .8})\nplt.title(\'Корреляционная матрица\')\nplt.show()'

In [24]:
#5.нормализация
from sklearn.linear_model import LinearRegression, Lasso, LassoCV, Ridge, RidgeCV

scaler = StandardScaler()

X_scaled_train = scaler.fit_transform(X_train)
X_scaled_valid = scaler.transform(X_valid)

#Тест
ID_col = X_test['ID']
X_test = X_test.drop(columns=['ID'])
X_scaled_test = scaler.transform(X_test)


In [ ]:
from sklearn.linear_model import Lasso, LassoCV, Ridge, RidgeCV
n_alphas = 250
ridge_alphas = np.logspace(-10, 15, n_alphas)
ridge_cv = RidgeCV(alphas=ridge_alphas, scoring="neg_mean_squared_error", cv=5)
ridge_cv.fit(X_scaled_train, y_train)
ridge_cv.alpha_

In [26]:
#6.обучение
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import Ridge, Lasso, ElasticNet
ridge_reg = Ridge(random_state=17, alpha=0.000001, max_iter=100000000, fit_intercept=True, solver='auto', tol=0.001)
ridge_reg.fit(X_scaled_train, y_train)
ridge_scaled_valid_pred = ridge_reg.predict(X_scaled_valid)
np.sqrt(mean_squared_error(ridge_scaled_valid_pred, y_valid))
#ridge_reg.fit(X_train, y_train)
#ridge_test_pred = ridge_reg.predict(X_test)
#1 1309446.791752681 все признаки дропнуты
#2 1309446.4427160455 TotalDebtToIncomeRatio оставил
#3 1309441.853532619 DebtToIncomeRatio добавил к последнему
# не менял CreditScore и История платежей / Предыдущие кредиты по умолчанию 
#4.	TotalAssets и NetWorth
#4 
#5.	BaseInterestRate и InterestRate :
#5 оставил
#6 в шестом пунте не менял
# Финиш
#8 Добавил в дроп NumberOfOpenCreditLine 1309429.1852870414
#Остались 4 и 7
# 7 взял разницу и дропнул два признака 1309428.811549499
# 4
#4.497480398047022
#4.497188244060865
#4.385616678342578
#4.384601337254873
#4.383944248220556
#4.383229530023397
#4.38123801150054
#4.378341493729471
#4.3551140088258
#4.137485532541913
#4.068619076560395
#3.891187822205368
#3.8754091467158687
#3.875391811595849
#3.8739981855877494
#3.872784842638023
#3.874712369063472
#3.8747153394372393
#3.887642340631539
#3.8898772806018416
#3.890470796791078
#3.858903691159399
#3.755014742795783
#3.7526069582799675
#3.372204225737339
#3.336178794392138
#3.280377485333526
#3.7376134917170156
#
#3.349657720976292
#3.3378580688248367
#3.283657445457522
#3.2829725678048627
#3.280098036175003
#3.2798840289557507
#3.2693559171397553
#3.256439775739252
#3.256254080243796
#3.216162066026205
#3.207619588299119
#3.201989498438146
#3.1813262229470594
#3.1794255238401807
#3.1770933941340487
#3.1768795332687527
#3.173493328567676
#3.1729923260604913
#3.1711614889322925
#3.1695278309447605
#3.167236064445428
#3.1618607942299346
#3.1593944963286034
#3.1554824352157285
#3.1526555812192876
#3.1527747019610355
#3.1336652717076454
#3.134280741996741
#3.134539329011779
#3.124874401617038
#3.1318125153168994
#3.1193506023061106
#3.0533219414569017
#3.023426621145234
#3.0096391699193634
#2.9979799635819604
#2.9868518582704593
#2.9870620055502
#2.9817292372403967
#2.959142426428376
#2.9502883831713436
#2.947151401308047
#2.9422929523447987
#2.9330891094236997
#2.880888857336444
#2.880888857483708
#2.8808153859935617
#2.876758149839732
#2.868116165497448
#2.868116165282157
#2.8610843453554424
#2.860709795081775
#2.860460962020005
#2.8590752621537683
#2.8522071039752683
#2.8450382222631805
#2.8456573609508102
#2.8455837975117633
#2.8455757343376757
#2.8451861936214033
#2.8450364665880206
#2.8428393504725844
#2.8427782976271057
#2.8425780222024866
#2.842532273949002
#2.842240486851297
#2.841988826682952
#2.8427829747496545
#2.8426903852979004
#2.8422946029288236
#2.8413145642430915
#2.8399698762082064
#2.8417641370701565
#2.8383364500186126
#2.8380562246176075
#2.834753565722771
#2.7954245806924662
#2.7846151672833233
#2.7845208405665693
#2.7836642628371804
#2.783124666702131
#2.752967786762735
#2.7525574726246007
#2.7501777137202548
#2.749890127292124
#2.749251397089738
#2.748047918537948
#2.7452647859579744
#2.743920315189408
#2.7435950950085823
#2.7425829320041824
#2.7423432687401776
#2.7423060346342862
#2.742155860212625
#2.7422660829187264
#2.740740634971721
#2.740740635067677
#2.740676910208875
#2.74067690936941
#2.7402687228357596
#2.7403617474444095
#2.740478266701014
#2.733200138187079
#2.7319528320598674
#2.7317025085794775
#2.731702349956722
#2.732247945733475
#2.730921128565146
#2.730639766290225
#2.7301862459912916
#2.7301355667420992
#2.7303969571318407
#2.7301203839177206
#2.7300648644758456
#2.7283546884790826
#2.728869497098975
#2.7286828249240838
#2.725537591390703
#2.7246429745745946
#2.7244767219847574
#2.7236429476294948
#2.7226143892398538
#2.7224838630438795
#2.721872072400283
#2.721845794685514
#2.721737955812755
#2.7212714813911023
#2.719706193785944
#2.719155890934601
#2.719080875669376
#2.7184726248718185
#2.714679404303234
#2.714275017371893
#2.7142298390376003

2.713136643166827

In [27]:
test_PRED = ridge_reg.predict(X_scaled_test)
ids = np.arange(test_PRED.shape[0]).reshape(-1, 1) 
data_reshaped = test_PRED.reshape(-1, 1) 
data_with_ids = np.hstack((ids, data_reshaped))  
df_out = pd.DataFrame(data_with_ids, columns=['ID', 'RiskScore'])
df_out.to_csv('submission.csv', index=False)